### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

# for openAI
import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

# Open source
groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'Your Shit'

In [1]:
!pip install langchain_groq

In [7]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001533DC4F250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001533DC4FC50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [8]:
# Translator

from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Bengali"),
    HumanMessage(content="I love you")
]

result=model.invoke(messages)
result

AIMessage(content='আমি তোমাকে প্রেম করি \n\n(Aami Tomake Premeri)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 46, 'total_tokens': 81, 'completion_time': 0.104893486, 'completion_tokens_details': None, 'prompt_time': 0.002307049, 'prompt_tokens_details': None, 'queue_time': 0.05075228, 'total_time': 0.107200535}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--a9747a34-a20a-4dd6-8b5f-0ebc552fdc1d-0', usage_metadata={'input_tokens': 46, 'output_tokens': 35, 'total_tokens': 81})

In [9]:
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()
parser.invoke(result)

'আমি তোমাকে প্রেম করি \n\n(Aami Tomake Premeri)'

In [10]:
# Using LCEL- chain the components

chain=model|parser
chain.invoke(messages)

'আমি তোমাকে ভালোবাসি'

In [11]:
# Prompt Templates (Effecient)
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [12]:
result=prompt.invoke({"language":"Hindi","text":"Love"})
result.to_messages()

[SystemMessage(content='Trnaslate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Love', additional_kwargs={}, response_metadata={})]

In [13]:
# Chaining together components with LCEL

chain=prompt|model|parser
chain.invoke({"language":"Hindi","text":"Love"})

'प्रेम'

In [14]:
!pip install streamlit